The purpose of this file is to generate yields for spring wheat and spring barley, along with nutrition, and spits out a couple plots and a csv file at the end of the file. I can't emphasize enough that you should take a look at Params.ods  to understand the inputs to the model. Params.ods is a spreadsheet, you can open it with excel or open office. Also reach out to morgan@allfed.info if you have questions about this code.

First, let's import a few dependencies. 
 * Params: Custom script which imports parameters from Params.ods file
     * Params.ods is as spreadsheet file located here: https://github.com/allfed/CropOpt/Params.ods
     * based on Adin's collaboration document here: https://docs.google.com/spreadsheets/d/1Mh7bmZYHypN7bSmejCyu4hkbolQ3MRANc3LhflwRZoA/edit#gid=0
 * Plotter: utility for displaying geopandas maps and plots
 * pandas, geopandas: Data analysis and GIS packages
     * some basic things you can do with pandas: https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html
     * some basic things you can do with geopandas: https://geopandas.org/getting_started/introduction.html
     
 * OutdoorGrowth: Custom class which contains several useful functions for outdoor growth calculations
     * if you want to understand what the model is doing to calculate yields, you need to read through OutdoorGrowth code, located here:https://github.com/allfed/CropOpt/tree/master/Modules

In [ ]:
import Params 
Params.importAll()
from Plotter import Plotter

import pandas as pd
import geopandas as gpd

import Modules.OutdoorGrowth
from Modules.OutdoorGrowth import OutdoorGrowth


Now, let's import a bunch of previously created geopandas datasets

In [ ]:
#total solar flux at surface , W/m^2
fsns=pd.read_pickle(Params.geopandasDataDir+'FSNS.pkl')

#surface temperature, K
ts=pd.read_pickle(Params.geopandasDataDir+'TS.pkl')

#Large-scale (stable) precipitation rate, including ice precipitate but not 
#snow, m/s
precl=pd.read_pickle(Params.geopandasDataDir+'PRECL.pkl')

#absolute humidity at surface, kg/kg
q=pd.read_pickle(Params.geopandasDataDir+'Q.pkl')

#crop area, Ha
growArea=pd.read_pickle(Params.geopandasDataDir+'CropGrowHectares.pkl')


Taking a look at sun, temperature, and rain, we see latitude and longitude, each month after the winter is defined in a column, and the cell geometry is displayed (in this case, a 2 degree by 2 degree square). The first few cells are all at the south pole, so it's unsurprising that they show strange values.

The default months in the Params.ods are May year 5- April year 6 (first and second year of simulation). 

The average temperature coefficient hottest consecutive months required to grow each crop in each region are used to estimate temperature and rainfall growth parameters.

There's an issue with the import for april of year six, for temperature. 

In [ ]:
fsns.head() #sun, W/m^2

In [ ]:
ts.head() #temp, kelvin

In [ ]:
precl.head() #rain, average m/s over the month

The grow area is shown below using hectares. Note it is only calculated for one month as it's assumed to be unchanging for now.

In [ ]:
growArea.head()

In [ ]:
q.head()

Let's also look at the values for some of the Params.ods variable values. It's important to be able to modify these if you want to be able to improve the cropOpt model. Make sure to restart the Jupyter kernel to reimport changes to params variables. The code below is looping through all the cells and all the months, so it might take a few minutes, depending on your machine.

Again, there appear to be issues with april of year 6 (second year of winter), and also possibly september year 5 (first year of winter).

In [ ]:
allMonths=Params.allMonths

#initialize object of outdoor growth class
outdoorGrowth=OutdoorGrowth()

In [ ]:
plotYields=True
yields=outdoorGrowth.estimateYields(ts,precl,growArea,plotYields)

Plots above are saved here: https://github.com/allfed/CropOpt/tree/master/Data/Figures

Below, we generate the remaining plots of the figures for all months of the simulation. This will take several minutes to run. To disable some of the plots, you can set the booleans plotGrowArea,plotsun,plotTemp,plotRain,or plotHum to false.

In [ ]:
plotSun=True
plotTemp=True
plotRain=True
plotHum=True

print('Producing Sun Figures')
for month in allMonths:
	title="Solar Flux at Surface, month "+month
	legendlabel="Solar Flux (W/m^2)"
	fn="SolarFlux"+month
	Plotter.plotMap(fsns,month,title,legendlabel,fn,plotSun)

print('Producing Temp Figures')
for month in allMonths:
	title="Surface (radiative) Temperature, month "+month
	legendlabel="Temperature (K)"
	fn="SurfaceTemp"+month
	Plotter.plotMap(ts,month,title,legendlabel,fn,plotTemp)

print('Producing Rain Figures')
for month in allMonths:
	title="Precipitation "+month
	legendlabel="Precipitation (m/s)"
	fn="Precipitation"+month
	Plotter.plotMap(precl,month,title,legendlabel,fn,plotRain)

print('Producing Humidity Figures')
for month in allMonths:
	title="Humidity "+month
	legendlabel="Humidity (kg/kg)"
	fn="Humidity"+month
	Plotter.plotMap(q,month,title,legendlabel,fn,plotHum)


Plots above are saved here: https://github.com/allfed/CropOpt/tree/master/Data/Figures

If you're interested in reproducing a CSV of many of the variables, you can call the function below:

In [ ]:
outdoorGrowth.saveLandSunHumRainCSV(ts,q,fsns,precl,growArea)